In [5]:
#this python program is for normalising and channel separation of image.
# here i separate channel within several folder for train and testing the svm classifier.


In [6]:
from rasterio.plot import show, show_hist
from google.colab import files,drive
from IPython import display
import matplotlib.pyplot as plt
import cv2
import rasterio
import seaborn as sns
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import tensorflow_io as tfio
import warnings
import pickle
import random

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from google.colab.patches import cv2_imshow
SEED = 1000
np.random.seed(SEED)
warnings.filterwarnings('ignore')
tf.random.set_seed(SEED)
tf.__version__




'2.4.1'

In [7]:
# !pip install rasterio
# !pip install tensorflow_io
# !pip install pickle-mixin

In [8]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
dir='/content/drive/MyDrive/Dataset'
category=['fl']
cll=['c_normalised','c_H_channel','c_E_channel']
fl=['f_normalised','f_H_channel','f_E_channel']
mcl=['m_normalised','m_H_channel','m_E_channel']

for i in category:
  path=os.path.join(dir,i)
  path2=os.path.join(dir,mcl[0])
  path3=os.path.join(dir,mcl[1])
  path4=os.path.join(dir,mcl[2])
  for image in os.listdir(path):
    imgpath=os.path.join(path,image)
    img = cv2.imread(imgpath,-1)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    io=240
    alpha=1
    beta=0.15
    heref=np.array([[0.5626,0.2159],[0.7201,0.8012],[0.4062,0.5581]])
    maxcref=np.array([1.9705,1.0308])
    h,w,c=img.shape
    img=img.reshape(-1,3)
    od=-np.log10((img.astype(np.float)+1)/io)
    from mpl_toolkits.mplot3d import Axes3D
    fig=plt.figure(figsize=(16,8))
    ax1=fig.add_subplot(121,projection='3d')
    ax1.scatter(img[:,0],img[:,1],img[:,2])
    ax2=fig.add_subplot(122,projection='3d')
    ax2.scatter(od[:,0],od[:,1],od[:,2])
    odhat=od[~np.any(od<beta,axis=1)]
    eigvals,eigvecs=np.linalg.eigh(np.cov(odhat.T))
    that=odhat.dot(eigvecs[:,1:3])
    phi=np.arctan2(that[:,1],that[:,0])
    minphi=np.percentile(phi,alpha)
    maxphi=np.percentile(phi,100-alpha)
    vmin=eigvecs[:,1:3].dot(np.array([(np.cos(minphi),np.sin(minphi))]).T)
    vmax=eigvecs[:,1:3].dot(np.array([(np.cos(maxphi),np.sin(maxphi))]).T)
    if vmin[0]>vmax[0]:
      he=np.array((vmin[:,0],vmax[:,0])).T
    else:
      he=np.array((vmax[:,0],vmin[:,0])).T
    y=np.reshape(od,(-1,3)).T
    c=np.linalg.lstsq(he,y,rcond=None)[0]
    maxc=np.array([np.percentile(c[:,0],99),np.percentile(c[1,:],99)])
    tmp=np.divide(maxc,maxcref)
    c2=np.divide(c,tmp[:,np.newaxis])
    inorm=np.multiply(io,np.exp(-heref.dot(c2))) 
    inorm[inorm>255]=254
    inorm=np.reshape(inorm.T,(h,w,3)).astype(np.uint8)
    H=np.multiply(io,np.exp(np.expand_dims(-heref[:,0],axis=1).dot(np.expand_dims(c2[0,:],axis=0))))
    H[H>255]=254,
    H=np.reshape(H.T,(h,w,3)).astype(np.uint8)
    E=np.multiply(io,np.exp(np.expand_dims(-heref[:,1],axis=1).dot(np.expand_dims(c2[1,:],axis=0))))
    E[E>255]=254
    E=np.reshape(E.T,(h,w,3)).astype(np.uint8)
    plt.imsave(os.path.join(path2,image),inorm)
    plt.imsave(os.path.join(path3,image),H)
    plt.imsave(os.path.join(path4,image),E)

  



In [ ]:
# dir='/content/drive/MyDrive/Dataset'
# category=['cll']
# cll=['c_normalised','c_H_channel','c_E_channel']
# fl=['f_normalised','f_H_channel','f_E_channel']
# mcl=['m_normalised','m_H_channel','m_E_channel']

# for i in category:
#   path=os.path.join(dir,i)
#   path2=os.path.join(dir,cll[0])
#   path3=os.path.join(dir,cll[1])
#   path4=os.path.join(dir,cll[2])
#   for image in os.listdir(path):
#     imgpath=os.path.join(path,image)
#     img = cv2.imread(imgpath,-1)
#     img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#     io=240
#     alpha=1
#     beta=0.15
#     heref=np.array([[0.5626,0.2159],[0.7201,0.8012],[0.4062,0.5581]])
#     maxcref=np.array([1.9705,1.0308])
#     h,w,c=img.shape
#     img=img.reshape(-1,3)
#     od=-np.log10((img.astype(np.float)+1)/io)
#     from mpl_toolkits.mplot3d import Axes3D
#     fig=plt.figure(figsize=(16,8))
#     ax1=fig.add_subplot(121,projection='3d')
#     ax1.scatter(img[:,0],img[:,1],img[:,2])
#     ax2=fig.add_subplot(122,projection='3d')
#     ax2.scatter(od[:,0],od[:,1],od[:,2])
#     odhat=od[~np.any(od<beta,axis=1)]
#     eigvals,eigvecs=np.linalg.eigh(np.cov(odhat.T))
#     that=odhat.dot(eigvecs[:,1:3])
#     phi=np.arctan2(that[:,1],that[:,0])
#     minphi=np.percentile(phi,alpha)
#     maxphi=np.percentile(phi,100-alpha)
#     vmin=eigvecs[:,1:3].dot(np.array([(np.cos(minphi),np.sin(minphi))]).T)
#     vmax=eigvecs[:,1:3].dot(np.array([(np.cos(maxphi),np.sin(maxphi))]).T)
#     if vmin[0]>vmax[0]:
#       he=np.array((vmin[:,0],vmax[:,0])).T
#     else:
#       he=np.array((vmax[:,0],vmin[:,0])).T
#     y=np.reshape(od,(-1,3)).T
#     c=np.linalg.lstsq(he,y,rcond=None)[0]
#     maxc=np.array([np.percentile(c[:,0],99),np.percentile(c[1,:],99)])
#     tmp=np.divide(maxc,maxcref)
#     c2=np.divide(c,tmp[:,np.newaxis])
#     inorm=np.multiply(io,np.exp(-heref.dot(c2))) 
#     inorm[inorm>255]=254
#     inorm=np.reshape(inorm.T,(h,w,3)).astype(np.uint8)
#     H=np.multiply(io,np.exp(np.expand_dims(-heref[:,0],axis=1).dot(np.expand_dims(c2[0,:],axis=0))))
#     H[H>255]=254,
#     H=np.reshape(H.T,(h,w,3)).astype(np.uint8)
#     E=np.multiply(io,np.exp(np.expand_dims(-heref[:,1],axis=1).dot(np.expand_dims(c2[1,:],axis=0))))
#     E[E>255]=254
#     E=np.reshape(E.T,(h,w,3)).astype(np.uint8)
#     plt.imsave(os.path.join(path2,image),inorm)
#     plt.imsave(os.path.join(path3,image),H)
#     plt.imsave(os.path.join(path4,image),E)

  

